# Understanding Removed Statements Dataset

Source of data: [GDrive | Removed Stataments of Wikidata | Feb 1 2021](https://drive.google.com/file/d/1TQP1rADdvhDjsvBpLzSE9Bx3n73wf-Md/view?usp=sharing)

Steps performed:
* Divide dataset into 2 halves - redirected and non-redirected. Redirected dataset has either node1 or node2 as redirected. But non-redirected has both node1, node2 not redirected


**Summary**

Removed Statements dataset has 76.5M removed statements. Out of these, 

## Redirects determination and division of dataset into 2 halves

* Since, redirects dataset was not present, a SPARQL query was run to determine all the redirects existing at the moment. This was done on Feb 19, 2021. This was executed using [SPARQL query](https://query.wikidata.org/). Query run was:
    ```
    SELECT ?old_node
    WHERE {
      ?old_node owl:sameAs ?new_node.
    }
    ```
* This has few lexemes as well which we don't need. So, I then ran the query:
    ```
    SELECT ?old_node
    WHERE {
      ?old_node owl:sameAs ?new_node.
      ?new_node rdf:type ontolex:LexicalEntry.
    }
    ```
* After removing the lexemes from the nodes file, a final redirected non-lexemes file was created with data from Feb 19, 2021: `data/SPARQL_redirects_non-lexemes.tsv`.
* Using this reduced dataset, I was able to determine in the removed_statements.tsv dataset, which nodes have been redirected - `../opAnalysis/removed_statements_redirects_basis_node1or2.tsv`. This has removed statements in which either node1 or node2 is redirected.
* After this, I am extracting the removed statements not present in this subset meaning it would correspond to all removed statements in neither node1 nor node2 is redirected - `../opAnalysis/removed_statements_both_nonredirects.tsv`

For this, I am using the following set of commands

In [2]:
import pandas as pd
import seaborn as sns

In [ ]:
# On the basis of SPARQL
!kgtk ifexists -i ../data/removed_statements.tsv\
    --filter-on ../data/SPARQL_redirects_non-lexemes.tsv \
    --filter-mode NONE \
    --input-keys node1 \
    --filter-keys id \
    -o ../opAnalysis/removed_statements_redirects_basis_node1.tsv
!kgtk ifnotexists -i ../data/removed_statements.tsv\
    --filter-on ../data/SPARQL_redirects_non-lexemes.tsv \
    --filter-mode NONE \
    --input-keys node1 \
    --filter-keys id \
    -o ../opAnalysis/removed_statements_nonredirects_basis_node1.tsv
!kgtk ifexists -i ../data/removed_statements.tsv\
    --filter-on ../data/SPARQL_redirects_non-lexemes.tsv \
    --filter-mode NONE \
    --input-keys node2 \
    --filter-keys id \
    -o ../opAnalysis/removed_statements_redirects_basis_node2.tsv
!kgtk ifnotexists -i ../data/removed_statements.tsv\
    --filter-on ../data/SPARQL_redirects_non-lexemes.tsv \
    --filter-mode NONE \
    --input-keys node2 \
    --filter-keys id \
    -o ../opAnalysis/removed_statements_nonredirects_basis_node2.tsv
!kgtk ifnotexists -i ../opAnalysis/removed_statements_redirects_basis_node1.tsv \
    --filter-on ../opAnalysis/removed_statements_redirects_basis_node2.tsv \
    -o ../opAnalysis/temp1.tsv
!kgtk cat -i ../opAnalysis/temp1.tsv \
    ../opAnalysis/removed_statements_redirects_basis_node2.tsv \
    -o ../opAnalysis/removed_statements_redirects_basis_node1or2.tsv
!kgtk ifnotexists -i ../data/removed_statements.tsv\
    --filter-on ../opAnalysis/removed_statements_redirects_basis_node1or2.tsv \
    -o ../opAnalysis/removed_statements_both_nonredirects.tsv

## P31 edges distribution

Now, we'll determine in this redirected dataset - `../opAnalysis/removed_statements_redirects_basis_node1or2.tsv`, how many of these are P31 edges and determine more stats on these

### For Redirected Removed Statements

In [ ]:
!kgtk --debug query -i ../opAnalysis/removed_statements_redirects_basis_node1or2.tsv \
    --match 'o: (a)-[:P31]->(b)' \
    --return 'b, count(distinct a)' \
    -o ../opAnalysis/removed_statements_redirects_P31_stats1.tsv

In [3]:
df1 = pd.read_csv('../opAnalysis/removed_statements_redirects_P31_stats1.tsv',sep='\t')
df1.columns = ['parent','count']
df1 = df1.sort_values(by=['count'],ascending=False)
df1 = df1.set_index('parent')
df1.head(15)

,count
parent,
Q4167836,526207
Q17329259,301359
Q5,222809
Q4167410,108583
Q13442814,101156
Q7187,88231
Q11266439,61007
Q4423781,53671
Q17143521,51581


### For non-redirected removed statements

In [ ]:
!kgtk --debug query -i ../opAnalysis/removed_statements_both_nonredirects.tsv \
    --match 'o: (a)-[:P31]->(b)' \
    --return 'b, count(distinct a)' \
    -o ../opAnalysis/removed_statements_nonredirects_P31_stats1.tsv

In [4]:
df1 = pd.read_csv('../opAnalysis/removed_statements_nonredirects_P31_stats1.tsv',sep='\t')
df1.columns = ['parent','count']
df1 = df1.sort_values(by=['count'],ascending=False)
df1 = df1.set_index('parent')

,count
parent,
P31,2381072
P17,357286
P1433,299464
P735,295778
P50,268412
P2860,243607
P625,227779
P106,185184
P131,183759


## Properties Distribution

### For redirected removed statements

In [ ]:
!kgtk --debug query -i ../opAnalysis/removed_statements_redirects_basis_node1or2.tsv \
    --match 'o: (a)-[r]->(b)' \
    --return 'r.label, count(distinct a)' \
    -o ../opAnalysis/removed_statements_redirects_props_dist.tsv

In [5]:
df1 = pd.read_csv('../opAnalysis/removed_statements_redirects_props_dist.tsv',sep='\t')
df1.columns = ['parent','count']
df1 = df1.sort_values(by=['count'],ascending=False)
df1 = df1.set_index('parent')
df1.head(15)

,count
parent,
P31,2381072
P17,357286
P1433,299464
P735,295778
P50,268412
P2860,243607
P625,227779
P106,185184
P131,183759


### For non-redirected removed statements

In [ ]:
!kgtk --debug query -i ../opAnalysis/removed_statements_both_nonredirects.tsv \
    --match 'o: (a)-[r]->(b)' \
    --return 'r.label, count(distinct a)' \
    -o ../opAnalysis/removed_statements_nonredirects_props_dist.tsv

In [ ]:
df1 = pd.read_csv('../opAnalysis/removed_statements_nonredirects_props_dist.tsv',sep='\t')
df1.columns = ['parent','count']
df1 = df1.sort_values(by=['count'],ascending=False)
df1 = df1.set_index('parent')
df1.head(15)

# Comparison Removed NR dataset with Qnodes, literals

In [2]:
!kgtk query -i ../opAnalysis/removed_statements_both_nonredirects.tsv \
    ../gdrive-kgtk-dump-2020-12-07/claims.tsv.gz \
    --match "r: (x)-[r]->(y), c: (x)-[s]->(z)" \
    --where "r.label = s.label" \
    --return 'x, r.label, y, s.label as newNode2Label, z as newNode2' \
    -o ../opAnalysis/removed_statements_both_nonredirects_new_vals.tsv

^C


In [3]:
!cat ../opAnalysis/removed_statements_both_nonredirects_new_vals.tsv

In [ ]:
!sed -i '1s/.*/node1\tlabel\tnode2\tnode2;newLabl\tnode2;nw/' removed_statements_both_nonredirects_str_new_vals.tsv